In [1]:
import os
import sys
import time
import cv2
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

device: cuda


In [2]:
# Adjust if your paths differ
PANECHO_ROOT = "/home/users/joshua04/141/PanEcho"
DATA_ROOT = "/home/users/joshua04/141/data/raw/echonet_pediatric/A4C"

sys.path.append(PANECHO_ROOT)
sys.path.append(os.path.join(PANECHO_ROOT, "src"))

In [3]:
from src.models import FrameTransformer

/home/users/joshua04/torch-gpu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]),])


In [5]:
def read_video_frames(path, n_frames=16):
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {path}")
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    idxs = np.linspace(0, max(length - 1, 0), n_frames).astype(int)
    frames = []
    i = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        if i in idxs:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(transform(frame))
        i += 1
    cap.release()
    while len(frames) < n_frames:
        frames.append(frames[-1])
    return torch.stack(frames, dim=1)  # (3, T, 224, 224)

In [6]:
class EchoNetVideoDataset(Dataset):
    def __init__(self, root, split="train", n_frames=16):
        df = pd.read_csv(os.path.join(root, "FileList.csv"))
        df = df[df["Split"] != 5] if split == "train" else df[df["Split"] == 5]
        self.df = df.reset_index(drop=True)
        self.root = root
        self.n_frames = n_frames
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.root, "Videos", row["FileName"])
        x = read_video_frames(path, self.n_frames)
        y = torch.tensor(row["EF"], dtype=torch.float32)
        return x, y

In [7]:
train_loader = DataLoader(
    EchoNetVideoDataset(DATA_ROOT, split="train"),
    batch_size=2,
    shuffle=True,
    num_workers=2,
    pin_memory=True)

val_loader = DataLoader(
    EchoNetVideoDataset(DATA_ROOT, split="val"),
    batch_size=2,
    shuffle=False,
    num_workers=2,
    pin_memory=True)

xb, yb = next(iter(train_loader))
print("batch x:", xb.shape, "batch y:", yb.shape)

batch x: torch.Size([2, 3, 16, 224, 224]) batch y: torch.Size([2])


In [8]:
backbone = FrameTransformer(
    arch="convnext_tiny",
    n_heads=8,
    n_layers=2,
    transformer_dropout=0.1,
    pooling="mean",
    clip_len=16)
model = nn.Sequential(
    backbone,
    nn.Linear(768, 1)).to(device)

print(model)

Sequential(
  (0): FrameTransformer(
    (encoder): ImageEncoder(
      (model): ConvNeXt(
        (features): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
            (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
          )
          (1): Sequential(
            (0): CNBlock(
              (block): Sequential(
                (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
                (1): Permute()
                (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
                (3): Linear(in_features=96, out_features=384, bias=True)
                (4): GELU(approximate='none')
                (5): Linear(in_features=384, out_features=96, bias=True)
                (6): Permute()
              )
              (stochastic_depth): StochasticDepth(p=0.0, mode=row)
            )
            (1): CNBlock(
              (block): Sequential(
                (0

In [9]:
for p in model[0].parameters():
    p.requires_grad = False

for p in model[1].parameters():
    p.requires_grad = True

print("Trainable parameters:")
for name, p in model.named_parameters():
    if p.requires_grad:
        print(" ", name)

Trainable parameters:
  1.weight
  1.bias


In [10]:
criterion = nn.MSELoss()   # EF regression
optimizer = optim.AdamW(
    model[1].parameters(),
    lr=1e-3,
    weight_decay=1e-4)

use_amp = (device == "cuda")
scaler = torch.amp.GradScaler("cuda", enabled=use_amp)

In [11]:
def run_one_epoch(model, loader, train=True):
    model.train(train)
    total_loss = 0.0
    total = 0
    for x, y in loader:
        x = x.to(device)
        y = y.to(device).view(-1, 1)
        if train:
            optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast("cuda", enabled=use_amp):
            pred = model(x)
            loss = criterion(pred, y)
        if train:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        total_loss += loss.item() * x.size(0)
        total += x.size(0)
    return total_loss / total

In [13]:
import math

num_epochs = 10  # start small

for epoch in range(num_epochs):
    train_loss = run_one_epoch(model, train_loader, train=True)
    val_loss   = run_one_epoch(model, val_loader,   train=False)

    print(
        f"Epoch {epoch:02d} | "
        f"train MSE: {train_loss:.1f} | "
        f"val MSE: {val_loss:.1f} | "
        f"val RMSE: {math.sqrt(val_loss):.2f}")

Epoch 00 | train MSE: 91.2 | val MSE: 134.4 | val RMSE: 11.60
Epoch 01 | train MSE: 85.9 | val MSE: 139.7 | val RMSE: 11.82
Epoch 02 | train MSE: 82.1 | val MSE: 121.6 | val RMSE: 11.03
Epoch 03 | train MSE: 79.5 | val MSE: 111.8 | val RMSE: 10.58
Epoch 04 | train MSE: 78.2 | val MSE: 134.3 | val RMSE: 11.59
Epoch 05 | train MSE: 75.7 | val MSE: 107.5 | val RMSE: 10.37
Epoch 06 | train MSE: 74.8 | val MSE: 106.3 | val RMSE: 10.31
Epoch 07 | train MSE: 73.2 | val MSE: 103.5 | val RMSE: 10.17
Epoch 08 | train MSE: 72.1 | val MSE: 103.7 | val RMSE: 10.18
Epoch 09 | train MSE: 71.4 | val MSE: 108.8 | val RMSE: 10.43


In [14]:
train_loss = run_one_epoch(model, train_loader, train=True)
val_loss   = run_one_epoch(model, val_loader,   train=False)

print(f"train loss: {train_loss:.3f}")
print(f"val   loss: {val_loss:.3f}")

train loss: 70.420
val   loss: 93.944
